<a href="https://colab.research.google.com/github/Martin-Teschke/IR_systems/blob/main/src/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
!wget http://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz

--2023-02-22 22:54:43--  http://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz
Resolving ir.dcs.gla.ac.uk (ir.dcs.gla.ac.uk)... 130.209.240.253
Connecting to ir.dcs.gla.ac.uk (ir.dcs.gla.ac.uk)|130.209.240.253|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 775144 (757K) [application/gzip]
Saving to: ‘cisi.tar.gz.1’

cisi.tar.gz.1       100%[===================>] 756.98K   914KB/s    in 0.8s    

2023-02-22 22:54:45 (914 KB/s) - ‘cisi.tar.gz.1’ saved [775144/775144]



In [2]:
!tar xvzf cisi.tar.gz

CISI.ALL
CISI.BLN
CISI.QRY
CISI.REL


In [14]:
import os
for dirname, _, filenames in os.walk('/content/files'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        with open(os.path.join(dirname, filename)) as f:
            line_count = 0
            id_set = set()
            for l in f.readlines():
                line_count += 1
                if filename == "CISI.REL":
                    id_set.add(l.lstrip(" ").split(" ")[0])
                elif l.startswith(".I "):
                    id_set.add(l.split(" ")[1].strip())
            print(f"{filename} : {len(id_set)} items, over {line_count} lines.")

/content/files/CISI.ALL
CISI.ALL : 1460 items, over 108747 lines.
/content/files/CISI.BLN
CISI.BLN : 0 items, over 86 lines.
/content/files/CISI.REL
CISI.REL : 76 items, over 3114 lines.
/content/files/CISI.QRY
CISI.QRY : 112 items, over 1525 lines.


In [16]:
with open('/content/files/CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
# print n lines
n = 5
for l in lines[:n]:
    print(l)

.I 1
.T 18 Editions of the Dewey Decimal Classifications
.A Comaromi, J.P.
.W The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad.
.X 1	5	1 92	1	1 262	1	1 556	1	1 1004	1	1 1024	1	1 1024	1	1


In [18]:
doc_set = {}
doc_id = ""
doc_text = ""
for l in lines:
    if l.startswith(".I"):
        doc_id = l.split(" ")[1].strip()
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.

# Print something to see the dictionary structure, etc.
print(f"Number of documents = {len(doc_set)}" + ".\n")
print(doc_set["1"]) # note that the dictionary indexes are strings, not numbers. 

Number of documents = 1460.

18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad. 


In [8]:
with open('/content/files/CISI.QRY') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = l.split(" ")[1].strip()
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""
    
# Print something to see the dictionary structure, etc.
print(f"Number of queries = {len(qry_set)}" + ".\n")
print(qry_set["3"]) # note that the dictionary indexes are strings, not numbers. 

Number of queries = 112.

What is information science?  Give definitions where possible.


In [61]:
!pip install rank_bm25
!pip install  nltk
from rank_bm25 import BM25Okapi
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
all_stopwords = stopwords.words('english')
all_stopwords.append('')

In [21]:
def data_prep(doc_set):

  def clean_corpus(input_string):
    return input_string.translate(str.maketrans('','', string.punctuation)).lower().lstrip(" ")

  doc_clean = {}
  # Retirando pontuação e caixa alta
  for key, value in doc_set.items():
    doc_clean[key] = clean_corpus(value)

  # Quebrando as frases em tokens
  tokenized_corpus = [doc.split(" ") for doc in doc_clean.values()]

  # Retirando stopwords
  tokens_without_sw = []
  lista = []
  for item in tokenized_corpus:
    for word in item:
      if word not in all_stopwords:
        lista.append(word)
    tokens_without_sw.append(lista)
    lista=[]

  return tokens_without_sw

In [23]:
tokens_corpus = data_prep(doc_set)
tokens_query = data_prep(qry_set)

In [26]:
bm25 = BM25Okapi(tokens_corpus)

In [80]:
resposta = bm25.get_top_n(tokens_query[0], tokens_corpus, n=1)

In [81]:
# tokens_corpus é uma lista, que é iniciada na posição 0 enquanto o doc_set é um dicionário na qual a primeira chave é a string 1.
# logo teremos que incrementar 1 na contagem quando mudamos do index do tokens_corpus para o doc_set onde localizamos o texto original.
# mesma lógica é aplicada para as queries.
tokens_corpus.index(resposta[0])

721

In [85]:
doc_set['722']

'Information Transfer Limitations of Titles of Chemical Documents Bottle, R.R. Seeley, C.R. Some methods of estimating the minimum amounts of information in a document not retrievable through its title are discussed.  An analysis of the information transferred by different types of keywords is helpful in planning search strategies, e.g., 30% of chemical substances mentioned in journal articles are not discernable in their titles even when broad class names are used as synonyms.  Patents have considerably less informative titles than journal articles.  In nuclear science, report titles are also less informative than those of journal articles, but the proportion of reports with completely uninformative titles is now only 10% of the 1957 value.  Titles in chemistry are more informative than those in most other fields, but the use of alerting and other services based on titles requires a good understanding of the underlying information transfer principles. '

In [86]:
qry_set['1']

'What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?'

In [71]:
data = [bm25.get_scores(query) for query in tokens_query]

In [73]:
matrix = pd.DataFrame(data=data, columns=['documento_'+key for key in doc_set.keys()], index=['query_'+key for key in qry_set.keys()])

In [90]:
matrix.T.sort_values(by=['query_1'], ascending=False)

,query_1,query_2,query_3,query_4,query_5,query_6,query_7,query_8,query_9,query_10,...,query_103,query_104,query_105,query_106,query_107,query_108,query_109,query_110,query_111,query_112
documento_722,26.821686,5.214518,1.910430,1.909822,1.984598,0.000000,5.913486,1.910430,6.824395,5.397583,...,4.446210,12.825051,4.874753,13.599420,38.926282,9.119012,15.620647,1.883492,11.506797,6.669314
documento_1299,23.662679,5.877079,2.010721,3.742470,0.717053,5.393582,7.704922,2.010721,5.877079,0.358526,...,0.000000,12.521579,0.358526,39.372799,5.471259,4.197091,13.666801,9.286922,2.896675,2.223290
documento_429,21.953168,4.262629,2.444881,1.758746,4.746422,0.000000,4.672965,1.729867,1.729867,2.768187,...,7.395412,20.371878,17.722563,14.667929,48.729563,6.348331,11.942513,6.721292,15.502582,11.374344
documento_1281,21.781078,0.320098,0.320098,0.000000,0.640196,0.000000,0.000000,0.320098,1.747394,5.407747,...,9.083865,13.521319,14.724690,23.531461,19.337303,4.201149,5.181719,0.320098,16.285344,10.771613
documento_759,21.345521,8.110040,0.000000,0.000000,0.000000,0.000000,7.110539,3.160579,3.949960,0.000000,...,1.930242,1.917258,2.732502,13.499331,9.128471,2.566350,11.462074,14.869839,15.346670,4.127733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
documento_615,0.000000,0.346084,0.346084,0.000000,7.539098,0.000000,10.310437,2.968113,6.898937,4.676257,...,4.023305,14.022402,3.174115,38.325162,22.715269,7.784728,15.678885,8.646381,10.222847,5.849051
documento_617,0.000000,1.217109,0.000000,0.000000,1.286511,0.000000,1.286511,1.286511,4.955691,1.286511,...,6.253500,4.772650,3.789254,14.366532,18.000820,26.668439,22.722172,5.763355,10.952809,12.022902
documento_618,0.000000,1.464918,1.653516,2.329096,0.441709,0.000000,2.329096,1.653516,0.220855,6.373341,...,1.940929,9.939484,5.544515,6.992243,2.897579,8.215089,25.185787,2.161784,8.591200,2.706562
documento_619,0.000000,3.158837,2.369910,0.000000,2.421074,0.000000,1.966789,4.336698,2.193931,2.193931,...,0.000000,13.029461,3.410517,27.541975,21.764139,7.706727,14.017804,24.164108,17.491944,4.770407
